<a href="https://colab.research.google.com/github/weso500/QMLIIOT/blob/main/MaprunsTEPQSVM_11_ZZlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install qiskit

In [11]:
pip install qiskit-machine-learning

In [12]:
pip install pyreadr

In [13]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [14]:
df_FaultFree = pyreadr.read_r(r'/content/drive/MyDrive/FDA Journal/TEP_FaultFree_Training.RData')['fault_free_training']
df_Faulty = pyreadr.read_r(r'/content/drive/MyDrive/FDA Journal/TEP_Faulty_Training.RData')['faulty_training']

In [15]:
DF = pd.concat([df_FaultFree,df_Faulty]) # Join both fault free and faulty dataset
DF.head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461


In [16]:
columns_to_drop = [
    'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_6', 'xmeas_10',
    'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_17', 'xmeas_20',
    'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28',
    'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34',
    'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40',
    'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6',
    'xmv_7', 'xmv_8', 'xmv_9', 'xmv_11'
]

DF = DF.drop(columns=columns_to_drop)
display(DF.head())

,faultNumber,simulationRun,sample,xmeas_5,xmeas_7,xmeas_8,xmeas_9,xmeas_11,xmeas_16,xmeas_18,xmeas_19,xmeas_21,xmeas_22,xmv_10
0,0.0,1.0,1,26.889,2704.3,74.863,120.41,80.044,3101.1,65.732,229.61,94.640,77.047,41.258
1,0.0,1.0,2,26.721,2705.0,75.000,120.41,80.078,3102.0,65.716,230.54,94.595,77.434,41.359
2,0.0,1.0,3,26.875,2706.2,74.771,120.42,80.220,3103.5,65.732,230.08,94.605,77.466,41.199
3,0.0,1.0,4,26.758,2707.2,75.224,120.39,80.305,3102.8,65.781,227.91,94.473,77.443,41.643
4,0.0,1.0,5,26.889,2705.1,75.388,120.39,80.064,3103.5,65.788,231.37,94.678,76.947,41.507


In [17]:

from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import PauliFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

dimension = 11
feature_map = ZZFeatureMap(11, reps=1, entanglement='linear')

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.kernel_approximation import Nystroem
from joblib import Parallel, delayed

# === Define helper for one trial ===
def run_trial(i):
    sc = StandardScaler()
    fault_free = DF[DF['faultNumber']==0].iloc[::50,3:]
    TEPtrain = fault_free.sample(n=200, random_state=i)
    TEP_test = fault_free.drop(TEPtrain.index).sample(n=50, random_state=i+100)
    TEPtrain = sc.fit_transform(TEPtrain)

    faulty = DF[DF['faultNumber']==1].iloc[::30,3:]
    faulty_samples = faulty.sample(n=10, random_state=i+200)
    TEPtest_with_faulty = pd.concat(
        [pd.DataFrame(TEP_test, columns=faulty_samples.columns), faulty_samples]
    )
    TEPtest_final = sc.transform(TEPtest_with_faulty)

    train_features = TEPtrain
    test_features  = TEPtest_final
    test_target    = np.concatenate((np.zeros(50), np.ones(10)))

    # --- Nyström approximation of the quantum kernel ---
    n_components = min(64, len(train_features)-1)
    Phi = Nystroem(
        kernel=lambda A,B=None: kernel.evaluate(x_vec=A, y_vec=B),
        n_components=n_components,
        random_state=i
    )
    Ztr = Phi.fit_transform(train_features)
    Zte = Phi.transform(test_features)

    # Compute approximate Gram matrices for precomputed OCSVM
    Ktr = Ztr @ Ztr.T
    Kte = Zte @ Ztr.T

    ocsvm = OneClassSVM(kernel='precomputed')
    ocsvm.fit(Ktr)
    scores = -ocsvm.decision_function(Kte)

    fpr, tpr, _ = metrics.roc_curve(test_target, scores, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print(auc)
    return auc

# === Run in parallel ===
aucs = Parallel(n_jobs=8)(delayed(run_trial)(i) for i in range(30))
print(f"\nMean AUC={np.mean(aucs):.3f} ± {np.std(aucs):.3f}")





Mean AUC=0.520 ± 0.108
